In [ ]:
import boto3
import os
s3 = boto3.client(
    's3',
    endpoint_url=os.environ['S3_ENDPOINT_URL'],
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY']
)
s3.download_file('datalake', 'deforestation/datatn/files/S2A_MSIL2A_20180129T101251_N0500_R022_T32TPS_20230904T215802.SAFE.zip', 'DATA/S2A_MSIL2A_20180129T101251_N0500_R022_T32TPS_20230904T215802.zip')
s3.download_file('datalake', 'deforestation/datatn/files/S2A_MSIL2A_20180422T102031_N0500_R065_T32TPS_20230915T161912.SAFE.zip', 'DATA/S2A_MSIL2A_20180422T102031_N0500_R065_T32TPS_20230915T161912.zip')
s3.download_file('datalake', 'deforestation/datatn/files/S2A_MSIL2A_20180926T101021_N0500_R022_T32TPS_20230729T180843.SAFE.zip', 'DATA/S2A_MSIL2A_20180926T101021_N0500_R022_T32TPS_20230729T180843.zip')

s3.download_file('datalake', 'deforestation/datatn/files/S2A_MSIL2A_20190328T102021_N0500_R065_T32TPS_20221115T132105.SAFE.zip', 'DATA/S2A_MSIL2A_20190328T102021_N0500_R065_T32TPS_20221115T132105.zip')
s3.download_file('datalake', 'deforestation/datatn/files/S2A_MSIL2A_20190626T102031_N0500_R065_T32TPS_20230723T161149.SAFE.zip', 'DATA/S2A_MSIL2A_20190626T102031_N0500_R065_T32TPS_20230723T161149.zip')
s3.download_file('datalake', 'deforestation/datatn/files/S2A_MSIL2A_20190904T102021_N0500_R065_T32TPS_20230703T073148.SAFE.zip', 'DATA/S2A_MSIL2A_20190904T102021_N0500_R065_T32TPS_20230703T073148.zip')


In [ ]:
!conda install gdal -y

### Deforestation Processing Chain

In [ ]:
import sys
import numpy as np
import sys, os, time, shutil, json
from os.path import abspath
import utils.filemanager as fm
from utils.S2L2A import L2Atile, getTileList
from utils.utils import _ndi
from datetime import datetime
from joblib import Parallel, cpu_count, delayed
from utils.utils import run_bfast_parallel, get_month_numbers, interpolate_for_year, interpolate_time_series, fuse_features, process_pixel
import utils.post_processing as pp


def deforestation(sensor, tilename, years, maindir, datapath, outpath):
    # Initialize logging and timer
    logging = {} 
    t_tot = time.time()

    # Check sensor type and get tile list
    if sensor == 'S2':
        tiledict = getTileList(datapath)
    else:
        raise IOError('Invalid sensor')
    
    keys = tiledict.keys()

    for k in keys:
        tileDatapath = tiledict[k]
        print(f"Reading Tile-{k}.")
        
        if sensor == 'S2':
            tile = L2Atile(maindir, tileDatapath)

        # Initialize empty storage for all years
        feature_NDVI_all = None
        feature_BSI_all = None
        all_dates = []    

        for y in years:
            # Set temporary path for the current year
            temppath = fm.joinpath(maindir, 'numpy', tilename)

            # Get features for the current year
            ts, _, _ = tile.gettimeseries(year=y, option='default')
            fn = [f for f in os.listdir(temppath)] 

            if len(ts) != 0:
                print(f'Extracting features for each image for year {y}:')
            
            # Get some information from data
            height, width = ts[0].feature('B04').shape
            geotransform, projection = fm.getGeoTIFFmeta(ts[0].featurepath()['B04'])
            ts_length = len(ts)

            ts = sorted(ts, key=lambda x: x.InvalidPixNum())[0:ts_length]
            totimg = len(ts)
            totfeature = 2
            feature_NDVI = np.empty((height, width, totimg))
            feature_BSI = np.empty((height, width, totimg))
            dates = []
               
            # Compute Index Statistics
            for idx, img in enumerate(ts):        
                print(f'.. {idx+1}/{totimg}      ', end='\r')   
                        
                # Compute NDVI and BSI indices
                b3 = img.feature('RED', dtype=np.float16)
                b4 = img.feature('nir', dtype=np.float16)
                b5 = img.feature('SWIR1', dtype=np.float16)
                date = img._metadata['date']
                dates.append(date)
    
                NDVI = _ndi(b4, b3)
                BSI = _ndi(b4, b5)
    
                # Mask for valid values (update if needed)
                fn = fn[1:]
                name = fn[idx]
                maskpath = fm.joinpath(temppath, name, 'MASK.npy')
                msk = np.load(maskpath)

                NDVI_mask = np.where(msk, np.nan, NDVI)
                BSI_mask = np.where(msk, np.nan, BSI)


                feature_NDVI[:, :, idx] = NDVI_mask
                feature_BSI[:, :, idx] = BSI_mask

                # Delete intermediate arrays to free memory
                del b3, b4, b5, NDVI, BSI, msk

            # Append yearly features and dates
            feature_NDVI_all = np.concatenate((feature_NDVI_all, feature_NDVI), axis=-1) if feature_NDVI_all is not None else feature_NDVI
            feature_BSI_all = np.concatenate((feature_BSI_all, feature_BSI), axis=-1) if feature_BSI_all is not None else feature_BSI
            all_dates.extend(dates)
            print(all_dates)

    #read the dates
    # Convert the date strings to datetime objects
    all_dates_datetime = [datetime.strptime(date, '%Y%m%d') for date in all_dates]
    
    # Separate dates based on the year
    dates_2018 = [date for date in all_dates_datetime if date.year == 2018]
    dates_2019 = [date for date in all_dates_datetime if date.year == 2019]

    
    #NDVI & BSI data
    ndvi_data = feature_NDVI_all
    bsi_data = feature_BSI_all
    
    height, width, _ = ndvi_data.shape
    interpolated_data = np.zeros((height, width, 24), dtype=np.float16)
    

    ndvi_data = np.asarray(ndvi_data, dtype=np.float16)
    bsi_data = np.asarray(bsi_data, dtype=np.float16)
    
    '''
    # Vectorized approach for fusion
    interpolated_ndvi = np.apply_along_axis(interpolate_time_series, 2, ndvi_data, dates_2018, dates_2019)
    interpolated_bsi = np.apply_along_axis(interpolate_time_series, 2, bsi_data, dates_2018, dates_2019)

    del ndvi_data, bsi_data
    
    # Fuse features across all pixels in a vectorized way
    interpolated_data = fuse_features(interpolated_ndvi, interpolated_bsi).astype(np.float16)
    
    '''
    
    # Run parallel processing fusion across pixels
    results = Parallel(n_jobs=-1, backend='loky')(
        delayed(process_pixel)(i, j) for i in range(height) for j in range(width)
    )


    # Store results back into the array
    for i, j, fused_pixel in results:
        interpolated_data[i, j, :] = fused_pixel

    # Reshape for BFAST
    totpixels = height * width
    fused_reshaped = interpolated_data.reshape((totpixels, 24))
   
    
    # Run BFAST
    startyear = int(years[0])
    endyear = int(years[-1]) 
    freq = 12 #monthly data
    nyear = endyear - startyear 
    years_np = np.arange(startyear, endyear+1)
    
    
    with Parallel(n_jobs=-1) as parallel:
        dates = bfast.r_style_interval((startyear, 1), (startyear + nyear, 365), freq).reshape(fused_reshaped.shape[1], 1)
        breaks, confidence = run_bfast_parallel(parallel, fused_reshaped, dates, freq)
          
    # Process results
    changemaps = breaks // freq
    accuracymaps = confidence
    changemaps = changemaps.reshape(height, width)
    accuracymaps = accuracymaps.reshape(height, width)
    
    
    
    changemaps_year = np.zeros_like(changemaps, dtype = int)
    for i, year in enumerate(years_np):
        changemaps_year[changemaps == i] = year
    
    
    # Remove isolated pixels
    updated_change_array, updated_probability_array = pp.remove_isolated_pixels(changemaps_year, accuracymaps)
    
    # Fill gaps and update probabilities
    final_change_array, final_probability_array = pp.fill_small_holes_and_update_probabilities(updated_change_array, updated_probability_array) 
    
    final_change_array = final_change_array.astype(float)
    final_probability_array = final_probability_array.astype(float)
    final_change_array[final_change_array ==0 ] = np.nan
    final_probability_array[final_probability_array ==0 ] = np.nan
    
    
    # Save output
    output_filename_change = fm.joinpath(outpath, "CD_2018_2019")
    output_filename_probability = fm.joinpath(outpath, "prob_2018_2019")
    fm.write_shapefile(changemaps_year, geotransform, projection, output_filename_change)
    fm.write_shapefile(accuracymaps, geotransform, projection, output_filename_probability)
    
                     
    output_filename_process_change = fm.joinpath(outpath,"CD_2018_2019_process")
    output_filename_process_probability = fm.joinpath(outpath,"prob_2018_2019_process")
    fm.write_shapefile(final_change_array, geotransform, projection, output_filename_process_change) 
    fm.write_shapefile(final_probability_array, geotransform, projection, output_filename_process_probability)                 
    
    
    print("Processing complete!")    
         

#PREPARE SOME TOOLBOX PARAMETERS
sensor = 'S2'
tilename = 'T32TPS'
years = ['2018','2019']
maindir = '/home/mkhatereh/'
datapath = '/home/mkhatereh/AIxPA/Code/Platform/DATA/'
outpath = '/home/mkhatereh/AIxPA/Code/Platform/OUTPUT/'
temppath = fm.joinpath(maindir, 'numpy')

deforestation(sensor, tilename, years, maindir, datapath, outpath)
